In [62]:
#imports
import numpy as np
from functools import cache

In [56]:
# read small input
input = open("./../../data/day_12/small_input.txt", 'r').readlines()

In [89]:
# read input
input = open("./../../data/day_12/input.txt", 'r').readlines()

In [90]:
#process input
records = []
groups = []
for line in input:
    line = line.strip()
    record, group = line.split()
    records.append(record)
    groups.append([int(x) for x in group.split(',')])


In [65]:
# part 1
## recursion
DEBUG = True
def compute_arrangements(record, i, group, g, rg):
    if i == len(record):
        if rg > 0:
            g = [x for x in g]
            g.append(rg)
        if g == group:
            if DEBUG:print(f"Anchor, return 1, g {g}, group {group}")
            return 1
        else:
            if DEBUG:print(f"Anchor, return 0, g {g}, group {group}")
            return 0
        
    if record[i] == ".": #machine is working correctly
        if DEBUG:print(f". at {i}")
        if rg > 0:
            g = [x for x in g]
            g.append(rg)
        if DEBUG:print(f"->continue recursion, rg {rg}, g {g}")
        return compute_arrangements(record, i+1, group, g, 0)
    
    elif record[i] == "#": #machine is broken
        if DEBUG:print(f"# at {i}")
        # if rg > 0 and (len(g) > (len(group)-1) or rg > group[len(g)]):
        #     if DEBUG:print(f"->stop recursion, rg {rg}, g {g}, group {group}")
        #     return 0 # stop recursion if the current group is too big
        # else:
        #     if DEBUG:print(f"->continue recursion, rg {rg}, g {g}")
        #     return compute_arrangements(record, i+1, group, g, rg)\
        return compute_arrangements(record, i+1, group, g, rg+1)
    
    elif record[i] == "?": #unknown
        if DEBUG:print(f"? at {i} split")
        #there could be some early stopping conditions here
        ng = g
        if rg > 0:
            #if ? is replaced with . and there is a running group, then the running group is finished
            ng = [x for x in g]
            ng.append(rg)

        if DEBUG:print(f"? at {i}->compute . with g {ng} and rg {0}")
        working = compute_arrangements(record, i+1, group, ng, 0)

        if DEBUG:print(f"? at {i}->compute # with g {g} and rg {rg+1}")
        broken = compute_arrangements(record, i+1, group, g, rg+1)
        
        return working + broken
    else:
        print("I should not be here")



In [49]:
#testing
g= []
test = compute_arrangements(records[2], 0, groups[2], g, 0) 
print(f"Test: {test}")
DEBUG = True

? at 0 split
? at 0->compute . with g [] and rg 0
# at 1
? at 2 split
? at 2->compute . with g [1] and rg 0
# at 3
? at 4 split
? at 4->compute . with g [1, 1] and rg 0
# at 5
? at 6 split
? at 6->compute . with g [1, 1, 1] and rg 0
# at 7
? at 8 split
? at 8->compute . with g [1, 1, 1, 1] and rg 0
# at 9
? at 10 split
? at 10->compute . with g [1, 1, 1, 1, 1] and rg 0
# at 11
? at 12 split
? at 12->compute . with g [1, 1, 1, 1, 1, 1] and rg 0
# at 13
? at 14 split
? at 14->compute . with g [1, 1, 1, 1, 1, 1, 1] and rg 0
Anchor, return 0, g [1, 1, 1, 1, 1, 1, 1], group [1, 3, 1, 6]
? at 14->compute # with g [1, 1, 1, 1, 1, 1] and rg 2
Anchor, return 0, g [1, 1, 1, 1, 1, 1, 2], group [1, 3, 1, 6]
? at 12->compute # with g [1, 1, 1, 1, 1] and rg 2
# at 13
? at 14 split
? at 14->compute . with g [1, 1, 1, 1, 1, 3] and rg 0
Anchor, return 0, g [1, 1, 1, 1, 1, 3], group [1, 3, 1, 6]
? at 14->compute # with g [1, 1, 1, 1, 1] and rg 4
Anchor, return 0, g [1, 1, 1, 1, 1, 4], group [1, 3, 1, 6]

In [64]:
# compute part 1
DEBUG = False
possible_arrangements_list = []
for i,record in enumerate(records):
    possible_arrangements = compute_arrangements(record, 0, groups[i], [], 0)
    possible_arrangements_list.append(possible_arrangements)
    print(f"Line {i+1} there are {possible_arrangements} possible arrangements.")

result = sum(possible_arrangements_list)
print(f"Part 1: In total, there are {result} possible arrangements.")


TypeError: unhashable type: 'list'

In [84]:
# compute part 1 with cache
DEBUG = False
possible_arrangements_list = []
for i,record in enumerate(records):
    possible_arrangements = compute_arrangements_cached(record, groups[i])
    possible_arrangements_list.append(possible_arrangements)
    print(f"Line {i+1} there are {possible_arrangements} possible arrangements.")

result = sum(possible_arrangements_list)
print(f"Part 1: In total, there are {result} possible arrangements.")


arrangements is 0
Line 1 there are 0 possible arrangements.
arrangements is 0
Line 2 there are 0 possible arrangements.
arrangements is 0
arrangements is 0
Line 3 there are 0 possible arrangements.
arrangements is 0
Line 4 there are 0 possible arrangements.
arrangements is 0
Line 5 there are 0 possible arrangements.
arrangements is 0
arrangements is 0
Line 6 there are 0 possible arrangements.
Part 1: In total, there are 0 possible arrangements.


In [95]:
# part 2
## dynamic programming

cache = {}
def compute_arrangements_cached(record, groups):
    #approach this by checking if combinations are possible instead of actually going through combinations
    if len(groups) == 0:
        return 1 if not '#' in record else 0
    
    min_space = sum(groups) + len(groups) - 1 #minimum space needed for groups and separators
    if min_space > len(record):
        return 0
    
    min_start = 0
    max_start = len(record) - min_space
    if '#' in record:
        max_start = min(max_start, record.index('#'))
    
    next_group, *rest_groups = groups

    arrangements = 0
    for start in range(min_start, max_start+1):
        end = start + next_group
        group = record[start:end]

        possible = (group.count("#") + group.count("?")) == len(group)

        separated = (end >= len(record)) or (record[end] == '.') or (record[end] == '?')
        if not possible or not separated:
            continue

        rest_record = record[end+1:]

        arguments = (rest_record, tuple(rest_groups))
        if arguments not in cache:
            cache[arguments] = compute_arrangements_cached(*arguments)
        arrangements += cache[arguments]

    return arrangements



In [96]:
# Part 2
## reprocess input
records2 = [f"{record}?{record}?{record}?{record}?{record}" for record in records]
groups2 = []
for group in groups:
    g = []
    for i in range(5):
        g.extend(group)
    groups2.append(g)

# compute part 2
DEBUG = False
possible_arrangements_list2 = []
cache = {}
for i,record in enumerate(records2):
    possible_arrangements = compute_arrangements_cached(record,groups2[i])
    possible_arrangements_list2.append(possible_arrangements)
    print(f"Line {i+1} there are {possible_arrangements} possible arrangements.")

result2 = sum(possible_arrangements_list2)
print(f"Part 2: In total, there are {result2} possible arrangements.")

Line 1 there are 1024 possible arrangements.
Line 2 there are 32768 possible arrangements.
Line 3 there are 15238392 possible arrangements.
Line 4 there are 32 possible arrangements.
Line 5 there are 4648830 possible arrangements.
Line 6 there are 8086554 possible arrangements.
Line 7 there are 1 possible arrangements.
Line 8 there are 1024 possible arrangements.
Line 9 there are 83610 possible arrangements.
Line 10 there are 17760104486 possible arrangements.
Line 11 there are 1 possible arrangements.
Line 12 there are 20681 possible arrangements.
Line 13 there are 18842 possible arrangements.
Line 14 there are 32 possible arrangements.
Line 15 there are 138366632 possible arrangements.
Line 16 there are 11584 possible arrangements.
Line 17 there are 22892490 possible arrangements.
Line 18 there are 53900 possible arrangements.
Line 19 there are 74357 possible arrangements.
Line 20 there are 2442310 possible arrangements.
Line 21 there are 8942790 possible arrangements.
Line 22 there 